# カーネルを使った強化学習

参考：
* [Kernel-Based Reinforcement Learning: A Finite-Time Analysis](https://arxiv.org/abs/2004.05599)

状態行動空間が連続のMDPではどのようにRLすればよいでしょうか？
今回はカーネルを使った強化学習を見てみます．

表記：
* $(x, a)$に定義された測度$P$と任意の関数$f$について，$P f(x, a)=P(\cdot \mid x, a) f=\int f(y) \mathrm{d} P(y \mid x, a)$
* $\mathcal{X}$と$\mathcal{A}$上に定義された測度：$\rho:(\mathcal{X} \times \mathcal{A})^2 \rightarrow \mathbb{R}_{\geq 0}$
* $\mathcal{X}$上の測度空間：$\left(\mathcal{X}, \mathcal{T}_{\mathcal{X}}\right)$．$\mathcal{T}_{\mathcal{X}}$はσ代数です．
* 報酬関数：$\left\{r_h\right\}_{h \in[H]}$は$\mathcal{X} \times \mathcal{A}$ to $[0,1]$の集合
* 遷移カーネル：$P=\left\{P_h\right\}_{h \in[H]}$．ここで，$P_h$は$(\mathcal{X} \times \mathcal{A}) \times \mathcal{T}_{\mathcal{X}}$ to $\mathbb{R}_{\geq 0}$なるカーネル．
* ベルマン方程式：$Q_h^*(x, a) \stackrel{\text { def }}{=} r_h(x, a)+\int_{\mathcal{X}} V_{h+1}^*(y) \mathrm{d} P_h(y \mid x, a)$
* リグレット：$\mathcal{R}(K) \stackrel{\text { def }}{=} \sum_{k=1}^K\left(V_1^*\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right)$


仮定：

---

* $(\mathcal{X}\times \mathcal{A})\times (\mathcal{X}\times \mathcal{A})$上の測度$\rho$は既知であるとする．
* 任意の$\left(x, x^{\prime}, a, a^{\prime}\right)$について，
$$
\rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]=\rho_{\mathcal{X}}\left(x, x^{\prime}\right)+\rho_{\mathcal{A}}\left(a, a^{\prime}\right)
$$
が成り立つような測度$\rho_{\mathcal{X}}$と$\rho_{\mathcal{A}}$が存在する．

---

報酬と遷移は次の意味でリプシッツ連続ですL

$$
\left|r_h(x, a)-r_h\left(x^{\prime}, a^{\prime}\right)\right| \leq \lambda_r \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

および

$$
W_1\left(P_h(\cdot \mid x, a), P_h\left(\cdot \mid x^{\prime}, a^{\prime}\right)\right) \leq \lambda_p \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

ここで，$W_1$は1-Wasserstein距離です．つまり，$W_1(\mu, \nu) \stackrel{\text { def }}{=}\sup _{f: \operatorname{Lip}(f) \leq 1} \int_{\mathcal{X}} f(y)(\mathrm{d} \mu(y)-\mathrm{d} \nu(y))$

---

このようなリプシッツ連続な報酬と遷移のもとでは最適Q値がリプシッツ連続になります．

---

**補題**

$L_h \stackrel{\text { def }}{=} \sum_{h^{\prime}=h}^H \lambda_r \lambda_p^{H-h^{\prime}}$
とする．このとき，

$$
\left|Q_h^*(x, a)-Q_h^*\left(x^{\prime}, a^{\prime}\right)\right| \leq L_h \rho\left[(x, a),\left(x^{\prime}, a^{\prime}\right)\right]
$$

が成立します．証明は簡単なので省略．

---



## カーネルUCBVI

$u, v \in \mathcal{X} \times \mathcal{A}$とします．関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$について，カーネル関数を

$$\psi_\sigma(u, v) \stackrel{\text { def }}{=} g(\rho[u, v] / \sigma)$$

として定めます．

---

**仮定**

関数$g: \mathbb{R}_{\geq 0} \rightarrow[0,1]$は微分可能，non-increasing，$g(4) > 0$を満たし，また，次を満たす定数$C^g_1, C^g_2$が存在します：

$$
g(z) \leq C_1^g \exp \left(-z^2 / 2\right) \text { and } \sup _z\left|g^{\prime}(z)\right| \leq C_2^g
$$

例えば$g(z)=\exp \left(-z^2 / 2\right)$のようなガウスカーネルが仮定を満たしています．

---

カーネルUCBVIではカーネル関数を使って過去のデータを重み付けし，報酬や遷移確率を推定します．

$(x, a)$ and $(s, h) \in[K] \times[H]$に対して，重み関数を

$$
\begin{aligned}
& w_h^s(x, a) \stackrel{\text { def }}{=} \psi_\sigma\left((x, a),\left(x_h^s, a_h^s\right)\right) \quad \text { and } \\
& \widetilde{w}_h^s(x, a) \stackrel{\text { def }}{=} \frac{w_h^s(x, a)}{\beta+\sum_{l=1}^{k-1} w_h^l(x, a)},
\end{aligned}
$$

とします．ここで$\beta > 0$は正則化項です．この重みを使って，

$$
\begin{aligned}
& \widehat{r}_h^k(x, a) \stackrel{\text { def }}{=} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) r_h^s, \\
& \widehat{P}_h^k(y \mid x, a) \stackrel{\text { def }}{=} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \delta_{x_{h+1}^s}(y) .
\end{aligned}
$$

のように報酬と遷移を計算します．
そして，推定した報酬と遷移を使って

$$
\widetilde{Q}_h^k(x, a)=\widehat{r}_h^k(x, a)+\widehat{P}_h^k V_{h+1}^k(x, a)+\mathrm{B}_h^k(x, a)
$$

のようにQ値を計算します．ここで$B$は探索用のボーナスです．
これを使って，

$$
Q_h^k(x, a) \stackrel{\text { def }}{=} \min _{s \in[k-1]}\left(\widetilde{Q}_h^k\left(x_h^s, a_h^s\right)+L_h \rho\left[(x, a),\left(x_h^s, a_h^s\right)\right]\right)
$$

のようにしてQ値の間隔を補完します．

---

**ボーナスについて**


$$
\mathbf{C}_h^k(x, a) \stackrel{\text { def }}{=} \beta+\sum_{s=1}^{k-1} w_h^s(x, a)
$$

を訪問回数の一般化とします．これを使って，ボーナスを

$$
\mathrm{B}_h^k(x, a) \approx \frac{H}{\sqrt{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+L_1 \sigma
$$

として定義します．

以上をまとめて，Kernel-UCBVIは次の処理を行います：

![KUCBVI](figs/Kernel-UCBVI-OptimisticQ.png)
![KUCBVI](figs/Kernel-UCBVI.png)



元論文はrlberryのTwinRoomで実験してますが，似たような簡単な環境を作ります．

**実装についてのコメント**
論文の実装では結局状態空間を離散化してる（representative stateってやつ？）[https://github.com/omardrwch/kernel_ucbvi_experiments/blob/main/algorithms/rs_kernelucbvi.py](https://github.com/omardrwch/kernel_ucbvi_experiments/blob/main/algorithms/rs_kernelucbvi.py)

結局やってることはテーブルMDPと変わらないかも．
一旦実装は保留しよう．

In [3]:
import numpy as np
from typing import NamedTuple, Optional, Callable

# [0, 0] ~ [1, 1]の部屋を動き回る環境を考えます
# [-0.1, -0.1] ~ [0.1, 0.1]の行動を考えます
S_low = np.array([0, 0])
S_high = np.array([1, 1])
A_low = np.array([-0.1, -0.1])
A_high = np.array([0.1, 0.1])


# 右上の方にいくと報酬が発生します
def rew_fn(s, a):
    if 0.8 <= s[0] <= 0.9 and 0.8 <= s[1] <= 0.9:
        return 1
    return 0


# 指定された行動のベクトルの方向に移動します
def tran_fn(s, a):
    noise = np.random.randn(2) * 0.01
    next_s = s + a + noise
    return next_s

# 初期状態です．固定します．
init_s = np.array([0.1, 0.1])

# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_low: np.ndarray
    S_high: np.ndarray
    A_low: np.ndarray
    A_high: np.ndarray
    H: int  # ホライゾン
    rew_fn: Callable
    tran_fn: Callable
    init_s: np.ndarray  # 初期分布


H = 30
mdp = MDP(S_low, S_high, A_low, A_high, H, rew_fn, tran_fn, init_s)

In [7]:
from tqdm import tqdm


def rho_s(s1, s2):
    return np.linalg.norm(s1 - s2)


def rho_a(a1, a2):
    return np.linalg.norm(a1 - a2)


def rho(sa1, sa2):
    s1, a1 = sa1
    s2, a2 = sa2
    return rho_s(s1, s2) + rho_a(a1, a2)


def kernel(sa1, sa2, sigma=0.1):
    # gaussianカーネルです
    z = rho(sa1, sa2) / sigma
    return np.exp(-z**2 / 2)


def compute_greedy_action(k, Ds, Da, Dns, Drew):
    for h in range(H):
        for m in range(k):
            Q

def KernelUCBVI(mdp: MDP, K: int):
    returns = []
    for k in tqdm(range(K)):
        s = mdp.init_s
        Ds = np.zeros(shape=(K, mdp.H, 2))  # state data
        Da = np.zeros(shape=(K, mdp.H, 2))  # action data
        Dns = np.zeros(shape=(K, mdp.H, 2))  # next state data
        Drew = np.zeros(shape=(K, mdp.H, 1))  # reward data

        # Q = optimistic_Q(...)
        ret = 0
        for h in range(H):
            a = compute_greedy_action(s, k, Ds, Da, Dns, Drew)
            a = np.random.rand(2)
            next_s = mdp.tran_fn(s, a)
            rew = mdp.rew_fn(s, a)
            Ds[k, h] = s
            Da[k, h] = a
            Dns[k, h] = next_s
            Drew[k, h] = rew
            s = next_s
            ret += rew

        returns.append(ret)

    return returns

K = 1000
returns = KernelUCBVI(mdp, K)

100%|██████████| 1000/1000 [00:00<00:00, 7735.14it/s]


## リグレット解析


このアルゴリズムについて，次のリグレットバウンドが成立します：

$$
\mathcal{R}(K) \leq \widetilde{\mathcal{O}}\left(H^2 \sqrt{\left|\mathcal{C}_\sigma\right| K}+L_1 K H \sigma+H^3\left|\mathcal{C}_\sigma\right|\left|\widetilde{\mathcal{C}}_\sigma\right|\right)
$$

ここで，$\left|\mathcal{C}_\sigma\right|$ と $\left|\widetilde{\mathcal{C}}_\sigma\right|$は$\left(\mathcal{X}\times \mathcal{A}, \rho\right)$と$\left(\mathcal{X}, \rho_{\mathcal{X}}\right)$の$\sigma$-coveringです．
また，$L_1$は最適Q値のリプシッツ定数です．

---

**証明**

$\widehat{P}_h^k(\cdot \mid x, a)$は今回はディラック測度の重み付き和担ってるので，$P_h(\cdot\mid x, a)$と$\widehat{P}_h^k(\cdot \mid x, a)$の距離をバウンドするのは難しいです．そこで，次の変形をします：

$$
\begin{aligned}
& \left|\left(\widehat{P}_h^k-P_h\right) V_{h+1}^*(x, a)\right| \\
& =\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) V_{h+1}^*\left(x_{h+1}^s\right)-P_h V_{h+1}^*(x, a)\right| \\
& \leq \underbrace{\left|\sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a)\left(V_{h+1}^*\left(x_{h+1}^s\right)-P_h V_{h+1}^*\left(x_h^s, a_h^s\right)\right)\right|}_{(\mathbf{A})} \\
& +\underbrace{\lambda_p L_{h+1} \sum_{s=1}^{k-1} \widetilde{w}_h^s(x, a) \rho\left[(x, a),\left(x_h^s, a_h^s\right)\right]}_{(\mathbf{B})}+\underbrace{\frac{\beta\left\|V_{h+1}^*\right\|_{\infty}}{\mathbf{C}_h^k(x, a)}}_{(\mathbf{C})} .
\end{aligned}
$$

Aの部分はマルチンゲール差分列であり，特殊なHoeffdingでバウンドできます.
BはLipschitz性を利用したことによるバイアスです．
Cは$\beta$のせいで出てくるバウンドです．
ゴニョゴニョすると，結局

$$
\left|\left(\widehat{P}_h^k-P_h\right) V_{h+1}^*(x, a)\right| \lesssim \frac{H}{\sqrt{\mathbf{C}_h^k(x, a)}}+\frac{\beta H}{\mathbf{C}_h^k(x, a)}+L_1 \sigma .
$$

が出てきます．あとは通常のOptimismを使った証明とだいたい同じです（多分）．